Initialization

In [1]:
# Import standard libraries for file handling and text processing
import os, pathlib, textwrap, glob

# # Load documents from various sources (URLs, text files, PDFs)
from langchain_community.document_loaders import UnstructuredURLLoader, TextLoader, PyPDFLoader

# # Split long texts into smaller, manageable chunks for embedding
from langchain.text_splitter import RecursiveCharacterTextSplitter

# # Vector store to store and retrieve embeddings efficiently using FAISS
from langchain.vectorstores import FAISS

# # Generate text embeddings using OpenAI or Hugging Face models
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceEmbeddings, SentenceTransformerEmbeddings

# # Use local LLMs (e.g., via Ollama) for response generation
from langchain.llms import Ollama

# # Build a retrieval chain that combines a retriever, a prompt, and an LLM
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# # Create prompts for the RAG system
from langchain.prompts import PromptTemplate

print("✅ Libraries imported! You're good to go!")

c:\Users\Extendad\miniconda3\envs\rag-chatbot\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Libraries imported! You're good to go!


Data Prepration

Ingest Source Modules

In [2]:
from langchain_community.document_loaders import PyPDFLoader, UnstructuredURLLoader
import glob;
row_docs = []
pdf_paths = glob.glob("data/Everstorm_*.pdf")

for pdf_path in pdf_paths: 
    loader = PyPDFLoader(pdf_path)
    doc = loader.load()
    row_docs.extend(doc)
print(row_docs)
print(f"Loaded {len(row_docs)} PDF pages from {len(pdf_paths)} files.")

Ignoring wrong pointing object 76 0 (offset 0)
Ignoring wrong pointing object 81 0 (offset 0)
Ignoring wrong pointing object 80 0 (offset 0)


[Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Payment_refund_and_security', 'source': 'data\\Everstorm_Payment_refund_and_security.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='FAQs  \n Which  payment  methods  do  you  accept?   Visa,  MasterCard,  AmEx,  Discover,  Apple  Pay,  Google  Pay,  PayPal,  Shop  Pay.  Installments  (US  \nonly),\n \nKlarna\n \nPay-in-4\n \n(selected\n \nEU\n \ncountries).\n  What  is  3-D  Secure  and  why  did  I  see  a  pop-up?    3-D  Secure  (also  “Verified  by  Visa”  /  “Mastercard  Identity  Check”)  adds  an  extra  one-time  code  \nfor\n \nEU\n \nPSD2\n \ncompliance.\n \nYour\n \nbank\n \ncontrols\n \nthat\n \npop-up.\n  Is  my  data  safe?   All  checkout  traffic  uses  TLS  1.3.  We  never  store  full  card  numbers.  Our  store  is  Level  1  \nPCI-DSS\n \ncompliant.\n  How  long  do  refunds  take?    We  issue  refunds  the  same  day  your  

2.1 - Load web pages

In [ ]:
from langchain_community.document_loaders import UnstructuredURLLoader
URLS = [
    "https://developer.bigcommerce.com/docs/store-operations/shipping",
    "https://developer.bigcommerce.com/docs/store-operations/orders/refunds",
]

try:
    row_docs.extend(UnstructuredURLLoader(URLS).load())
except Exception as e:
    print("⚠️  Web fetch failed, using offline copies:", e)
    raw_docs = []
 
    print(f"Loaded {len(raw_docs)} offline documents.")

print(row_docs)

[Document(metadata={'producer': 'Skia/PDF m138 Google Docs Renderer', 'creator': 'PyPDF', 'creationdate': '', 'title': 'Payment_refund_and_security', 'source': 'data\\Everstorm_Payment_refund_and_security.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='FAQs  \n Which  payment  methods  do  you  accept?   Visa,  MasterCard,  AmEx,  Discover,  Apple  Pay,  Google  Pay,  PayPal,  Shop  Pay.  Installments  (US  \nonly),\n \nKlarna\n \nPay-in-4\n \n(selected\n \nEU\n \ncountries).\n  What  is  3-D  Secure  and  why  did  I  see  a  pop-up?    3-D  Secure  (also  “Verified  by  Visa”  /  “Mastercard  Identity  Check”)  adds  an  extra  one-time  code  \nfor\n \nEU\n \nPSD2\n \ncompliance.\n \nYour\n \nbank\n \ncontrols\n \nthat\n \npop-up.\n  Is  my  data  safe?   All  checkout  traffic  uses  TLS  1.3.  We  never  store  full  card  numbers.  Our  store  is  Level  1  \nPCI-DSS\n \ncompliant.\n  How  long  do  refunds  take?    We  issue  refunds  the  same  day  your  

2.2 - Chunk the text

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter 
chunks = []
text_splitter =  RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=30 )
chunks =  text_splitter.split_documents(row_docs)
print(len(chunks))

42


Build a Retriver

3.1 - Load a model to generate embeddings


In [5]:

from langchain.embeddings import HuggingFaceEmbeddings

embeddingModel = HuggingFaceEmbeddings(model_name = "thenlper/gte-small")
embedding_vector =  embeddingModel.embed_documents([d.page_content for d in chunks ])
print(len(embedding_vector[0]))
len(embedding_vector)


384


42

3.2 - Build a vector database

In [6]:

from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS  

text_embedding_pairs = zip([d.page_content for d in chunks ], embedding_vector)  
vectore_store = FAISS.from_embeddings(text_embedding_pairs, embeddingModel)
# vectore_store.similarity_search_by_vector()
response = vectore_store.similarity_search(
    query= "what is Eligibility Checklist", 
    k= 8
)
print(response)

[Document(id='6873b585-aa73-4a1a-8c03-9d5ec87e2d30', metadata={}, page_content='Checklist    ●  Unworn,  unwashed,  no  odors,  tags  attached    ●  Original  shoe  box  (footwear)  placed  inside  outer  carton    ●  Electronics  (power-banks,  headlamps)  unopened  unless  faulty   How  to  Start    ●  Visit  everstorm.example/returns  →  enter  order  #  and  email.    ●'), Document(id='2d1b32ba-79a3-4061-89dd-c06ef1dbc11a', metadata={}, page_content='(selected\n \nEU\n \ncountries).\n  What  is  3-D  Secure  and  why  did  I  see  a  pop-up?    3-D  Secure  (also  “Verified  by  Visa”  /  “Mastercard  Identity  Check”)  adds  an  extra  one-time  code  \nfor\n \nEU\n \nPSD2\n \ncompliance.\n \nYour\n \nbank\n \ncontrols\n \nthat\n \npop-up.'), Document(id='e31250b6-6877-4810-8039-40d53b5ee979', metadata={}, page_content='klarna.com\n \nor\n \ntheir\n \nmobile\n \napp.\n  Why  was  my  card  declined?    Common  reasons:  ZIP  mismatch,  CVV  typo,  foreign  card  blocked  for  cros

4.1 - Install ollama and serve gemma3

In [7]:
from langchain.llms import Ollama
llm_model = Ollama(model = "gemma3:1b")
response =  llm_model.invoke("hello ollama")

C:\Users\Extendad\AppData\Local\Temp\ipykernel_21232\1176389820.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm_model = Ollama(model = "gemma3:1b")


Build a RAG

5.1 - Define a system prompt

In [8]:
SYSTEM_TEMPLATE = """
You are a **Customer Support Chatbot**. Use only the information in CONTEXT to answer.
If the answer is not in CONTEXT, respond with “I'm not sure from the docs.”

Rules:
1) Use ONLY the provided <context> to answer.
2) If the answer is not in the context, say: "I don't know based on the retrieved documents."
3) Be concise and accurate. Prefer quoting key phrases from the context.
4) When possible, cite sources as [source: source] using the metadata.

CONTEXT:
{context}

USER:
{question}
"""

In [16]:
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history" , return_messages= True)

prompt = PromptTemplate.from_template(template=SYSTEM_TEMPLATE)

q =  ConversationalRetrievalChain.from_llm(
    llm_model, 
    memory = memory, 
    retriever=vectore_store.as_retriever(),
    combine_docs_chain_kwargs={"prompt": prompt}
)
# # questionGeneratorChain = LLMChain(llm_model , prompt); 
# combine_docs_chain = StuffDocumentsChain(...)



In [18]:
resposne = q({"question" : "Which payment methods do you accept? "})
response

'Hello there! How can I help you today? 😊 \n\nIs there anything you’d like to talk about, or anything I can do for you?'

In [19]:
q({"question" : "Which payment methods do you accept? "})

{'question': 'Which payment methods do you accept? ',
 'chat_history': [HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept Visa, MasterCard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept the following payment methods: Visa, Mastercard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept the following payment methods: Visa, Mastercard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={})],
 'answer': 'We accept the following payment methods: Visa, Mastercard, AmEx, Disc

In [20]:
q({"question" : "till me about first payment method "})

{'question': 'till me about first payment method ',
 'chat_history': [HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept Visa, MasterCard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept the following payment methods: Visa, Mastercard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept the following payment methods: Visa, Mastercard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='till me about first payment method ', additional_kwargs={}, 

In [21]:
q({"question" : "what is the first payment method you list"})

{'question': 'what is the first payment method you list',
 'chat_history': [HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept Visa, MasterCard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept the following payment methods: Visa, Mastercard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept the following payment methods: Visa, Mastercard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='till me about first payment method ', additional_kwarg

In [22]:
q({"question" : "explain for me first one"})

{'question': 'explain for me first one',
 'chat_history': [HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept Visa, MasterCard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept the following payment methods: Visa, Mastercard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept the following payment methods: Visa, Mastercard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='till me about first payment method ', additional_kwargs={}, response_me

In [23]:
q({"question" : "How long will delivery take for a standard order, and where can I track my package once it ships?"})

{'question': 'How long will delivery take for a standard order, and where can I track my package once it ships?',
 'chat_history': [HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept Visa, MasterCard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept the following payment methods: Visa, Mastercard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='Which payment methods do you accept? ', additional_kwargs={}, response_metadata={}),
  AIMessage(content='We accept the following payment methods: Visa, Mastercard, AmEx, Discover, Apple Pay, Google Pay, PayPal, Shop Pay.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content